# Программы с фильтром по жанру и времени выхода
Пример выгрузки выходов программ определенного жанра (телесериал) с ограничением по ЦА и и времени начала

## Описание задачи и условий расчета
- Период: 19.09.2022 - 02.10.2022
- ЦА: Все 18+, города 100+
- Место просмотра: Все места (Дом+Дача)
- Каналы: ПЕРВЫЙ КАНАЛ, РОССИЯ 1, ТВ ЦЕНТР, НТВ, СТС, ТНТ, ТВ-3, РЕН ТВ, ПЯТЫЙ КАНАЛ
- Программы: жанр - телесериал, Время начала – 18:50-23:00
- Статистики: Rtg%, Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники
Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
#Обратимся к каталогу жанров программ
cats.get_tv_program_type()

#id жанра Телесериал - 1

In [ ]:
# Отберем нужные нам каналы в справочнике телекомпаний и выведем их id
cats.get_tv_company(name=[
    "ПЕРВЫЙ КАНАЛ (СЕТЕВОЕ ВЕЩАНИЕ)",
    "РОССИЯ 1 (СЕТЕВОЕ ВЕЩАНИЕ)",
    "ТВ ЦЕНТР (СЕТЕВОЕ ВЕЩАНИЕ)",
    "НТВ (СЕТЕВОЕ ВЕЩАНИЕ)",
    "СТС (СЕТЕВОЕ ВЕЩАНИЕ)",
    "ТНТ (СЕТЕВОЕ ВЕЩАНИЕ)",
    "ТВ-3 (СЕТЕВОЕ ВЕЩАНИЕ)",
    "РЕН ТВ (СЕТЕВОЕ ВЕЩАНИЕ)",
    "ПЯТЫЙ КАНАЛ (СЕТЕВОЕ ВЕЩАНИЕ)"])['id'].tolist()

In [ ]:
# В словаре демографических атрибутов найдем переменную Население 100+
cats.find_tv_property('100+')

# В условия расчетета передадим cube100Plus100Minus

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2022-09-19', '2022-10-02')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА - все 18+, города 100+
basedemo_filter = 'age >= 18 and cube100Plus100Minus = 1'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы
company_filter = 'tvCompanyId IN (1859, 1862, 1867, 1870, 1858, 1861, 1872, 1877, 1873)'

# Указываем фильтр программ: жанр - Телесериал, время начала с 18:50 до 23:00  
program_filter = 'programTypeId = 1 and programStartTime >= 185000 and programStartTime <= 230000'

# Фильтр блоков
break_filter = None

# Фильтр роликов
ad_filter = None

# Указываем список срезов
slices = ['programSpotId', #Программа ID выхода, обязательный атрибут! 
          'researchDate', #Дата, обязательный атрибут! 
          'programName', #Программа
          'tvCompanyName', #Телекомпания
          'researchWeekDay', #День недели
          'programStartTime', #Программа время начала
          'programFinishTime', #Программа время окончания
         ]
# Указываем список статистик для расчета
statistics = ['RtgPer', 'Share']

# Задаем опции расчета
options = {
    "kitId": 1
}

## Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_simple_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, options=options)

# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_simple_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task))
df

In [ ]:
#ОПЦИОНАЛЬНО
#Задаем порядок столбцов
df = df[['programSpotId','tvCompanyName','programName','researchDate','researchWeekDay',
         'programStartTime','programFinishTime','RtgPer', 'Share']]

In [ ]:
df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('03_genre_filter')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)